In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import gc
import time
import xgboost
print("XGBoost version:",xgboost.__version__)

/glade/work/zhonghua/personal_clone_name/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


XGBoost version: 1.1.1


In [2]:
# get the dataframe work raw nc data
def get_df(file, vari):
    file = path+year+"_"+month+".nc"
    ds = xr.open_dataset(file)
    da_dict = {}
    # T
    da_dict["T"] = ds["T"].isel(lev =-1).drop(["lev"],axis=1)
    # RELUHM 
    RH = ds["RELHUM"].isel(lev =-1).drop(["lev"],axis=1)/100.0
    da_dict["RELHUM"] = xr.where(RH <= 1, RH, 1)
    # merge T and RELUHM
    ds_f = xr.merge([da_dict["T"],da_dict["RELHUM"]])
    del RH, da_dict 
    gc.collect()

    # SZA
    ds_f["SZA"] = np.radians(ds.SZA)
    # O3
    ds_f["O3_SRF"] = ds["O3"].isel(lev=-1)*1.0e9

    # gas in ppb
    for gas in ['SOAG_SRF','DMS_SRF','H2SO4_SRF','H2O2_SRF','SO2_SRF']:
        ds_f[gas] = ds[gas]*1.0e9

    # density and get aerosol in ug/m3
    ds_f["RHO_CLUBB"] = ds["RHO_CLUBB"].isel(ilev=-1)
    ds_f['Mass_bc']=(ds['bc_a1_SRF']+ds['bc_a4_SRF'])*ds_f["RHO_CLUBB"]*1.0e9
    ds_f['Mass_dst']=(ds['dst_a1_SRF']+ds['dst_a2_SRF'])*ds_f["RHO_CLUBB"]*1.0e9
    ds_f['Mass_ncl']=(ds['ncl_a1_SRF']+ds['ncl_a2_SRF'])*ds_f["RHO_CLUBB"]*1.0e9
    ds_f['Mass_pom']=(ds['pom_a1_SRF']+ds['pom_a4_SRF'])*ds_f["RHO_CLUBB"]*1.0e9
    ds_f['Mass_so4']=(ds['so4_a1_SRF']+ds['so4_a2_SRF'])*ds_f["RHO_CLUBB"]*1.0e9
    ds_f['Mass_soa']=(ds['soa_a1_SRF']+ds['soa_a2_SRF'])*ds_f["RHO_CLUBB"]*1.0e9

    # drop density
    ds_f = ds_f.drop(["RHO_CLUBB"])
    
    # convert to dataframe
    df = ds_f.to_dataframe().reset_index()[["lat","lon","time"]+vari]
    
    return df

# apply the ML and return to netcdf
def predict_chi_ls(df, vari, chi_ls):
    df_copy = df.copy()
    for chi in chi_ls:
        XGBreg_load=pickle.load(open("./xgb_model/"+chi+".dat","rb"))
        print("start to get",chi)
        time_s=time.time()
        df_copy[chi]=XGBreg_load.predict(df[vari])
        
        print("before clip")
        print("min:",float(df_copy[chi].min()),",max:",float(df_copy[chi].max()))
        print("after clip") 
        df_copy[chi]=df_copy[chi].clip(0,1)
        print("min:",float(df_copy[chi].min()),",max:",float(df_copy[chi].max()))
                                         
        print("It took",time.time()-time_s,"to get chi \n")
        del XGBreg_load
        gc.collect()
    
    time_s=time.time()
    df_xr = df_copy.set_index(["time","lat","lon"]).to_xarray()
    print("It took",time.time()-time_s,"to get dataset")
    
    return df_xr

In [ ]:
path = "/glade/scratch/zhonghua/mam4_cesm_raw/"
save_path="/glade/scratch/zhonghua/mam4_cesm_pred/"
chi_ls = ["chi_b","chi_c","chi_h"]
vari = ["DMS_SRF","H2O2_SRF","H2SO4_SRF","O3_SRF","SO2_SRF","SOAG_SRF",
        'Mass_so4','Mass_bc','Mass_ncl','Mass_dst','Mass_pom','Mass_soa',
        "T", "RELHUM", "SZA"]

for year in ["2011"]:
    print("start year:",year)
    for i in tqdm(range(1,13)):
        month=str(i).zfill(2)
        print("start the month:",month)
        file = path+year+"_"+month+".nc"
        df = get_df(file,vari)
        print("The shape is:", df.shape)
        df_pred = predict_chi_ls(df, vari, chi_ls)
        df_pred.to_netcdf(save_path+year+"_"+month+".nc")
        
        del df, df_pred
        gc.collect()
        print("\n")
    print("\n")